In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from numpy import log
from statsmodels.tsa.stattools import adfuller
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn import datasets
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA
from datetime import datetime
%matplotlib inline
sns.set()

from statsmodels.tsa.stattools import acf, pacf
#import statsmodels.tsa.statespace.sarimax as SARIMAX

/Users/mehrnazakbari/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


ModuleNotFoundError: No module named 'seaborn'

## 1_ Data Collection

### Transactions' DB

In [ ]:
trans_df = pd.read_csv('/Users/mehrnazuser/Documents/Lavoro/BlueRockLogistics/Data/Trans_BLR.csv', usecols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], sep =';', encoding = 'utf-8')
trans_df.head()

### Masters' DB

In [ ]:
depot_add = pd.read_excel('/Users/mehrnazuser/Documents/Lavoro/BlueRockLogistics/Data/masterdata20190918.xlsx', sheet_name = 'Depot Account_Addreses')
#depot_add.head()

## 2_ Data Cleansing

In [ ]:
#Type_List = ['COL', 'RET']
#trans_df = trans_df.loc[trans_df['T_DOC_TYPE'].isin(Type_List)]
trans_df = trans_df.loc[trans_df['T_DOC_TYPE'] == 'COL']

trans_df['T_EFFECTIVE_DATE'] = pd.to_datetime(trans_df['T_EFFECTIVE_DATE'])
trans_df['Date'] = trans_df.apply(lambda r:r['T_EFFECTIVE_DATE'].date(), axis = 1)
trans_df = trans_df.dropna(subset = ['Date'])

trans_df = trans_df[['Date', 'T_DOC_TYPE', 'T_SENDING_ID', 'T_RTI_NAME', 'T_ACT_HAND_OUT_QTY']]
trans_df = trans_df.reset_index()
trans_df = trans_df.drop('index', axis = 1)
trans_df.head()

In [ ]:
depot_add = depot_add[['name', 'code', 'country']]
#depot_add.head()

## 3_ Data Integration

In [ ]:
trans_full = trans_df.join(depot_add.set_index('code'), on = 'T_SENDING_ID')
trans_full = trans_full.drop(['T_SENDING_ID', 'T_DOC_TYPE'], axis =1)
trans_full.head()

## 4_ Filter (Country = Italy)

In [ ]:
trans_full = trans_full.loc[trans_full['country'] == 'IT']
trans_full = trans_full.sort_values('Date', ascending = True)
#trans_full['name'].unique()

## 5_ Functions

In [ ]:
def RTI(df):
    rti = df['T_RTI_NAME'].unique()
    n = 0
    for x in range(len(rti)):
        n += 1
        print('\n' + str(n) + '_ ' + rti[x])
    if len(rti) > 1:
        n += 1
        print('\n' + str(n) + '_ ' + 'All Containers')
    
    #return df

In [ ]:
def RTI_choice(df):
    rt = df['T_RTI_NAME'].unique()
    rti_list = []
    q = raw_input('\n Do you want to choose all RTIs? (Yes/No) : ')
    if q == 'Yes':
        df1 = df
    elif q == 'No':
        n = int(raw_input('\n How many RTI(s) you want to choose: '))
        if n >= 1:
            for x in range (1, n):
                rti = int(raw_input('\n Enter your choice '+ str(x) + ': '))
                rti_list.append(rt[rti])
                x += 1

        else:
            rti = raw_input('\n Enter the number of your choice: ')
            rti_list = rt[rti]

                
    else:
        print('\n Invalid input.')

    return rti_list
    

In [ ]:
def RTI_choice1(df):
    rtt = df['T_RTI_NAME'].unique()
    rti = (int(input('\n Choose a container number from the list above: '))) - 1
    if rti< len(rtt):
        print('\n Your choice is: ' + rtt[rti])
        df = df.loc[df['T_RTI_NAME'] == rtt[rti]]
    elif rti == len(rtt):
        print('\n You chose All RTIs in Germany.')

    return df

In [ ]:
def periodic_plot(df, p):
    
    depots = df['name'].unique()   
    if (p == 'Weekly') :
        #df = df.resample('1W').sum()
        for x in range(len(depots)):
            b = df.groupby(['Date', 'name']).sum().unstack('name', fill_value=0)
            b.resample('1W').plot(figsize=(20,10), linewidth=5, fontsize=20)
            plt.title('RTI Weekly Growth in Diverse Depots', fontsize=20)
            plt.xlabel('Weekly', fontsize=20)
            plt.show()
            return b
            
            
            
            #df_depot = df.loc[df['name'] == depots[x].encode('utf-8')]
            #df_depot = df_depot.resample('1W').sum()
            #df_depot.plot(figsize=(20,10), linewidth=5, fontsize=20)
            #plt.title(str(x+1) + '_ ' + depots[x], fontsize=20)
            #plt.xlabel(p , fontsize=20);
            #x += 1
            #return df            
        
        
        
    elif (p == 'Monthly'):
        for x in range(len(depots)):
            df_depot = df[df['name'] == depots[x]]
            df_depot = df_depot.resample('1M').sum()
            x += 1
    elif (p == 'Quarterly'):
        for x in range(len(depots)):
            df_depot = df[df['name'] == depots[x]]
            df_depot = df_depot.resample('1Q').sum()
            x += 1
    else:
        print('\n Invalid input.')
    
    return df

In [ ]:
def plot_prova(c):
    b = c.groupby(['Date', 'name']).resample('1M').sum().unstack('name', fill_value=0)
    b.plot(figsize=(20,10), linewidth=5, fontsize=20)
    plt.title('RTI Monthly Growth in Diverse Depots', fontsize=20)
    plt.xlabel('Monthly', fontsize=20)
    plt.show()
    return b

## 6_ State of the Art

In [ ]:
print('\n List of Containers in Italy: ')
RTI(trans_full)

In [ ]:
c = RTI_choice1(trans_full)
c = c.dropna()
depots = c['name'].unique()

c.Date = pd.to_datetime(c.Date)
c = c.sort_values('Date', ascending = True)
tutto = c
c = c.groupby(['Date', 'name'], as_index = False).sum()
c = c.set_index('Date')
#c.head()

In [ ]:
depo = c['name'].unique()
print('\n The list of depo(s) the chosen RTI is collected from is as follows:')
#b = bytes(depo)
#str(b, encoding = 'utf-8')
for x in range(len(depo)):
    print('\n' + str(x+1) + '_ ' + depo[x])
    c.loc[c['name'] == depo[x]]


## 7_ History of Transactions in every Italian Inventory

### 7.1_ Daily basis

In [ ]:
deps_q = c.groupby(['Date', 'name'])['T_ACT_HAND_OUT_QTY'].sum()
deps_q.unstack('name', fill_value=0).plot(figsize=(20,10), linewidth=5, fontsize=20)
plt.title('History of Transactions in Italian Inventories', fontsize=20)
plt.xlabel('Date (Daily basis)', fontsize=16)
plt.ylabel('Quantity', fontsize=16)
plt.show()

### 7.2_ Weekly basis

In [ ]:
c1 = c.groupby(['Date', 'name'])['T_ACT_HAND_OUT_QTY'].sum().unstack('name', fill_value=0)
c_week = c1.resample('1W').sum()
c_week.plot(figsize=(20,10), linewidth=5, fontsize=20)
plt.title('History of Transactions in Italian Inventories', fontsize=20)
plt.xlabel('Date (Weekly basis)', fontsize=16)
plt.ylabel('Quantity', fontsize=16)
plt.show()
#c_week

### 7.3_ Seasonal basis

In [ ]:
# c1 = c.groupby(['Date', 'name'])['T_ACT_HAND_OUT_QTY'].sum().unstack('name', fill_value=0)
deps_q_3m = c1.resample('3M').sum()
deps_q_3m.plot(figsize=(20,10), linewidth=5, fontsize=20)
plt.title('History of Transactions in Italian Inventories', fontsize=20)
plt.xlabel('Date (Seasonal basis)', fontsize=16)
plt.ylabel('Quantity', fontsize=16)
plt.show()

## 8_ All Transactions in Italy

In [ ]:
tutto = tutto.groupby('Date').sum()
tutto.plot(figsize=(12, 5), linewidth=5, fontsize=14, color = 'black')
plt.title('Container Transactions in Italy', fontsize = 14)
plt.xlabel('Date (Daily basis)', fontsize = 14)
plt.ylabel('Quantity', fontsize = 14)
plt.show()

### 8.1_ Seasonal Aggregation

In [ ]:
ita_3m = tutto.resample('3M').sum()
ita_3m.plot(figsize=(12, 5), linewidth=5, fontsize = 14, color = 'black')
plt.title('Container Transactions in Italy', fontsize = 14)
plt.xlabel('Date (Seasonal basis)', fontsize = 14)
plt.ylabel('Quantity', fontsize = 14)
plt.show()

#### 8.1.1_ AutoCorrelation Function

In [ ]:
#depot = c['name'].nunique()
ita_3m_q = ita_3m[['T_ACT_HAND_OUT_QTY']].dropna()
#tutti = tutto_week[['T_ACT_HAND_OUT_QTY']].dropna()
ita_3m_acf = acf(ita_3m_q, nlags=10)
plt.figure(figsize=(12, 5))
plt.plot(ita_3m_acf, marker="o", color = 'black')
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(depo)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(depo)),linestyle='--',color='gray')
plt.title('ACF_Italy', fontsize = 14)
plt.xlabel('number of lags', fontsize = 14)
plt.ylabel('Autocorrelation', fontsize = 14)
plt.tight_layout()

#### 8.1.2_ Partial AutoCorrelation Function

In [ ]:
ita_3m_pacf = pacf(ita_3m_q, nlags = 10 , method='ols')
#Plot PACF:
plt.figure(figsize=(12, 5))
plt.plot(ita_3m_pacf, marker='+', color = 'black')
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(depo)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(depo)),linestyle='--',color='gray')
plt.title('PACF_Italy', fontsize=14)    
plt.xlabel('Number of lags', fontsize = 14)
plt.ylabel('Partial Autocorrelation', fontsize = 14)
plt.tight_layout()

#### 8.1.3_ Prediction

In [ ]:
X = ita_3m.values
size = int(len(X) * 0.8)
train_3m, test_3m = X[0:size], X[size:len(X)]
history = [x for x in train_3m]

In [ ]:
import statsmodels.api as sm

my_order = (1,0,1)
my_seasonal_order = (3,0,1,3)
model = sm.tsa.statespace.SARIMAX(train_3m, order=my_order, seasonal_order=my_seasonal_order)
model_fit = model.fit(disp = False)

In [ ]:
yhat = model_fit.forecast(steps = 5)
yhat

In [ ]:
test_3m

In [ ]:
plt.figure(figsize=(12, 5))
pyplot.plot(test_3m, color = 'black', label = 'testing_set')
pyplot.plot(yhat, color = 'red', label = 'forecast()')
plt.legend()
plt.title('Prediction vs. Testing-set', fontsize=14)    
plt.xlabel('Number of lags', fontsize = 14)
plt.ylabel('Quantity', fontsize = 14)

#### 8.1.4_ Accuracy

In [ ]:
mse_Italy_3m = mean_squared_error(test_3m, yhat)
print('MSE: ' + str(mse_Italy_3m))

### 8.2_ Differencing the Time-Series (Seasonal Aggregation)

In [ ]:
ita_3m_diff = ita_3m.diff()
ita_3m_diff.plot(figsize=(12,5), linewidth=5, fontsize=14, color = 'black')
plt.title('Differenced RTI Growth in Italy', fontsize=14)
plt.xlabel('Date (Seasonal basis)', fontsize=14)
plt.show()

#### 8.2.1_ ACF

In [ ]:
ita_3m_diff_q = ita_3m_diff[['T_ACT_HAND_OUT_QTY']].dropna()
#tutti = tutto_week[['T_ACT_HAND_OUT_QTY']].dropna()
ita_3m_diff_acf = acf(ita_3m_diff_q, nlags=10)
plt.figure(figsize=(12, 5))
plt.plot(ita_3m_diff_acf, marker="o", color = 'black')
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(depo)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(depo)),linestyle='--',color='gray')
plt.title('ACF_Differenced Data', fontsize = 14)
plt.xlabel('number of lags', fontsize = 14)
plt.ylabel('Autocorrelation', fontsize = 14)
plt.tight_layout()

#### 8.2.2_ PACF

In [ ]:
ita_3m_diff_pacf = pacf(ita_3m_diff_q, nlags = 10 , method='ols')
#Plot PACF:
plt.figure(figsize=(12, 5))
plt.plot(ita_3m_diff_pacf, marker='+', color = 'black')
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(depo)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(depo)),linestyle='--',color='gray')
plt.title('PACF_Differenced Data', fontsize=14)    
plt.xlabel('Number of lags', fontsize = 14)
plt.ylabel('Partial Autocorrelation', fontsize = 14)
plt.tight_layout()

#### 8.2.3_ Prediction

In [ ]:
import statsmodels.api as sm

my_order = (1,1,1)
my_seasonal_order = (3,1,1,3)
model_diff = sm.tsa.statespace.SARIMAX(train_3m, order=my_order, seasonal_order=my_seasonal_order)
model_diff_fit = model_diff.fit(disp = False)

In [ ]:
yhat_diff = model_diff_fit.forecast(steps = 5, typ = 'levels')
yhat_diff

In [ ]:
plt.figure(figsize=(12, 5))
pyplot.plot(test_3m, color = 'black', label = 'testing_set')
pyplot.plot(yhat_diff, color = 'gold', label = 'diff. forecast()')
pyplot.plot(yhat, color = 'red', label = 'forecast()')
plt.legend()
plt.title('Comparison Among Testing-set & Prediction Methods', fontsize=14)    
plt.xlabel('Number of lags', fontsize = 14)
plt.ylabel('Quantity', fontsize = 14)

#### 8.2.4_ Accuracy

In [ ]:
mse_Italy_diff_3m = mean_squared_error(test_3m, yhat_diff)

if(mse_Italy_diff_3m < mse_Italy_3m):
    print("The YELLOW line is more accurate")
    print("\nDifferencing makes the prediction more Accurate.")
elif(mse_Italy_diff_3m > mse_Italy_3m):
    print("The RED line is more accurate")
    print("\nThus, differencing does NOT make the prediction more Accurate.")

### 8.3_ Aggregation on 6 months' basis

In [ ]:
ita_6m = tutto.resample('6M').sum()
ita_6m.plot(figsize=(12, 5), linewidth=5, fontsize = 14, color = 'brown')
plt.title('Container Transaction in Italy', fontsize = 14)
plt.xlabel('Date (aggregation of every 6 months)', fontsize = 14)
plt.show()

#### 8.3.1_ ACF

In [ ]:
ita_6m_q = ita_6m[['T_ACT_HAND_OUT_QTY']].dropna()
#tutti = tutto_week[['T_ACT_HAND_OUT_QTY']].dropna()
ita_6m_acf = acf(ita_6m_q, nlags=10)
plt.figure(figsize=(12, 5))
plt.plot(ita_6m_acf, marker="o", color = 'brown')
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(depo)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(depo)),linestyle='--',color='gray')
plt.title('ACF_Italy', fontsize = 14)
plt.xlabel('number of lags', fontsize = 14)
plt.ylabel('Autocorrelation', fontsize = 14)
plt.tight_layout()

#### 8.3.2_ PACF

In [ ]:
ita_6m_pacf = pacf(ita_6m_q, nlags = 10 , method='ols')
#Plot PACF:
plt.figure(figsize=(12, 5))
plt.plot(ita_6m_pacf, marker='+', color = 'brown')
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(depo)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(depo)),linestyle='--',color='gray')
plt.title('PACF_Italy', fontsize = 14)    
plt.xlabel('Number of lags', fontsize = 14)
plt.ylabel('Partial Autocorrelation', fontsize = 14)
plt.tight_layout()

#### 8.3.3_ Prediction

In [ ]:
Z = ita_3m.values
size = int(len(Z) * 0.8)
train_6m, test_6m = Z[0:size], Z[size:len(Z)]
history = [z for z in train_6m]

In [ ]:
my_order = (2,0,0)
my_seasonal_order = (2,0,0,6)
model_6m = sm.tsa.statespace.SARIMAX(train_6m, order=my_order, seasonal_order=my_seasonal_order)
model_6m_fit = model_6m.fit(disp = False)

In [ ]:
yhat_6m = model_6m_fit.forecast(steps = 5, typ = 'levels')
yhat_6m

In [ ]:
arima6m_fit = ARIMA(train_6m, order=my_order).fit(disp = False)
arima_yhat = arima6m_fit.forecast(steps = 5)[0]
arima_yhat

In [ ]:
plt.figure(figsize=(12, 5))
pyplot.plot(test_6m, color = 'brown', label = 'testing_set')
#pyplot.plot(yhat_diff, color = 'gold')
pyplot.plot(yhat_6m, color = 'red', label = 'forecast()')
pyplot.plot(arima_yhat, color = 'blue', label = 'ARIMA')
plt.legend()
plt.title('Comparison Among Testing-set, Forecast() & ARIMA', fontsize=14)    
plt.xlabel('Number of lags', fontsize = 14)
plt.ylabel('Quantity', fontsize = 14)

#### 8.3.4_ Accuracy

In [ ]:
mse_arima_6m = mean_squared_error(test_6m, arima_yhat)
mse_yhat_6m = mean_squared_error(test_6m, yhat_6m)

if(mse_arima_6m < mse_yhat_6m):
    print("The BLUE line is more accurate.")
elif(mse_arima_6m > mse_yhat_6m):
    print("The RED line is more accurate.")

### 8.4_ Differencing the Time-Series (6m)

In [ ]:
ita_6m_diff = ita_6m.diff()
ita_6m_diff.plot(figsize=(12, 5), linewidth=5, fontsize=14, color = 'brown')
plt.title('Differenced RTI Growth in Italy', fontsize=14)
plt.xlabel('Date (aggregation of 6m)', fontsize=14)
plt.ylabel('Quantity', fontsize = 14)
plt.show()

#### 8.4.1_ Prediction

In [ ]:
my_order_diff = (1,1,0)
my_seasonal_order = (1,1,0,6)
model_diff6m = sm.tsa.statespace.SARIMAX(train_6m, order=my_order, seasonal_order=my_seasonal_order)
model_diff6m_fit = model_diff6m.fit(disp = False)

In [ ]:
yhat_6m_diff = model_6m_fit.forecast(steps = 5, typ = 'levels')
yhat_6m_diff

In [ ]:
plt.figure(figsize=(12, 5))
pyplot.plot(test_6m, color = 'brown', label = 'testing_set')
pyplot.plot(yhat_diff, color = 'gold', label = 'diff. forecast()')
pyplot.plot(yhat_6m, color = 'red', label = 'forecast()')
pyplot.plot(arima_yhat, color = 'blue', label = 'ARIMA')
plt.legend()
plt.title('Comparison Among Testing-set(6m) & Diverse Forecasting Methods', fontsize=14)    
plt.xlabel('Number of lags', fontsize = 14)
plt.ylabel('Quantity', fontsize = 14)

#### 8.4.2_ Accuracy

In [ ]:
mse_yhat_diff_6m = mean_squared_error(test_6m, yhat_diff)

if (min(mse_arima_6m, mse_yhat_6m, mse_yhat_diff_6m) == mse_arima_6m):
    print("the BLUE line indicating ARIMA method is more accurate.")
    
if (min(mse_arima_6m, mse_yhat_6m, mse_yhat_diff_6m) == mse_yhat_6m):
    print("the RED line indicating forecast() method is more accurate.")

if (min(mse_arima_6m, mse_yhat_6m, mse_yhat_diff_6m) == mse_yhat_diff_6m):
    print("the YELLOW line indicating forecast() method on differenced data is more accurate.")